In [ ]:
from groundedPL.codificacion import ToPropositionalLogic
from groundedPL.logic_tester import LogicTester
from groundedPL.logUtils import LogUtils
from typing import List, Tuple
from itertools import product
import random

parser = ToPropositionalLogic()


triplas = product(range(8), range(8), range(4))

for x, y, n in triplas:
    parser.parse(f'En({x}, {y}, {n})')

In [ ]:
NUM_FILAS = 8
NUM_COLUMNAS = 8


def barco_h(x: int, y: int, n: int) -> List[List[Tuple[int, int]]]:
    """
    Genera todos los barcos horizontales de tamaño `n` que pasan por (x, y).
    
    Parámetros:
        x, y: Coordenadas de la casilla (0-7)
        n: Tamaño del barco (1-4)
    
    Retorna:
        Lista de barcos, donde cada barco es una lista de tuplas (fila, columna)
    """
    if not (0 <= x < NUM_FILAS and 0 <= y < NUM_COLUMNAS):
        return []
    if not (1 <= n <= 4):
        return []
    barcos = []
    # Rango de posibles posiciones iniciales horizontales
    inicio_min = max(0, x - n + 1)
    inicio_max = x
    
    for inicio_x in range(inicio_min, inicio_max + 1):
        casillas = [(inicio_x + i, y) for i in range(n)]
        # Verificar que todas las casillas estén dentro del tablero
        if all(0 <= cx < NUM_FILAS for cx, _ in casillas):
            barcos.append(casillas)
    
    return barcos


def barco_v(x: int, y: int, n: int) -> List[List[Tuple[int, int]]]:
    """
    Genera todos los barcos verticales de tamaño `n` que pasan por (x, y).
    
    Parámetros:
        x, y: Coordenadas de la casilla (0-7)
        n: Tamaño del barco (1-4)
    
    Retorna:
        Lista de barcos, donde cada barco es una lista de tuplas (fila, columna)
    """
    if not (0 <= x < NUM_FILAS and 0 <= y < NUM_COLUMNAS):
        return []
    if not (1 <= n <= 4):
        return []
    barcos = []
    # Rango de posibles posiciones iniciales verticales
    inicio_min = max(0, y - n + 1)
    inicio_max = y
    
    for inicio_y in range(inicio_min, inicio_max + 1):
        casillas = [(x, inicio_y + j) for j in range(n)]
        # Verificar que todas las casillas estén dentro del tablero
        if all(0 <= cy < NUM_COLUMNAS for _, cy in casillas):
            barcos.append(casillas)
    
    return barcos

# Regla general: Barcos alineados horizontal o verticalmente Y todas las casillas entre inicio y fin de un barco deben estar ocupadas
def regla_general():
    condiciones = []
    for x in range(NUM_FILAS):
        for y in range(NUM_COLUMNAS):
            for n in range(1, 5):
                configuraciones = []
                # Horizontal
                for barco in barco_h(x, y, n):
                    clausulas = (f"En({u},{v},{n})" for (u, v) in barco)
                    configuraciones.append(LogUtils.Ytoria_str(clausulas))

                # Vertical
                for barco in barco_v(x, y, n):
                    clausulas = (f"En({u},{v},{n})" for (u, v) in barco)
                    configuraciones.append(LogUtils.Ytoria_str(clausulas))

                # Si hay alguna configuración, construimos la implicación
                if configuraciones:
                    antecedente = f"En({x},{y},{n})"
                    consecuente = LogUtils.Otoria_str(configuraciones)
                    implicacion = f"({antecedente} -> {consecuente})"
                    condiciones.append(implicacion)

    return LogUtils.Ytoria_str(condiciones)

#Regla 4: No superposición de barcos
def crear_regla_4():
    """
    Regla corregida: No superposición de barcos de diferentes tamaños
    """
    condiciones = []
    for x in range(NUM_FILAS):
        for y in range(NUM_COLUMNAS):
            for n in range(1, 5):
                for m in range(n + 1, 5):  # Solo pares únicos, evitar duplicados
                    # Si hay un barco de tamaño n en (x,y), no puede haber uno de tamaño m
                    exclusion = f"-(En({x},{y},{n}) & En({x},{y},{m}))"
                    condiciones.append(exclusion)
    
    return LogUtils.Ytoria_str(condiciones)



# Genera automáticamente 4 barcos (tamaños 1-4) sin solapamientos.
def generar_barcos_aleatorios() -> List[Tuple[int, int, int]]:
    """
    Coloca un barco de cada tamaño en {1,2,3,4} de forma aleatoria,
    sin solapamientos y respetando los límites del tablero.
    Devuelve una lista de tuplas (x, y, n) representando las casillas-clave.
    """
    occupied = set()
    barcos_shape = []
    # Iterar tamaños de barco
    for n in range(1, 5):
        candidatos = []
        # Generar formas horizontales posibles
        for i in range(NUM_FILAS):
            for j in range(NUM_COLUMNAS - n + 1):
                shape = [(i, j + k) for k in range(n)]
                if not any(cell in occupied for cell in shape):
                    candidatos.append(shape)
        # Generar formas verticales posibles
        for i in range(NUM_FILAS - n + 1):
            for j in range(NUM_COLUMNAS):
                shape = [(i + k, j) for k in range(n)]
                if not any(cell in occupied for cell in shape):
                    candidatos.append(shape)
        if not candidatos:
            raise ValueError(f"No hay espacio para colocar barco de tamaño {n}")
        elección = random.choice(candidatos)
        barcos_shape.append((elección, n))
        occupied.update(elección)

    barcos = []
    for shape, n in barcos_shape:
        x, y = random.choice(shape)
        barcos.append((x, y, n))
    return barcos

# Generar barcos aleatorios en lugar de definir la lista manualmente
barcos = generar_barcos_aleatorios()

# Imprimir configuración aleatoria
print("Barcos generados aleatoriamente:")
for x, y, n in barcos:
    print(f"Barco de tamaño {n} en ({x}, {y})")


# Regla para los barcos en tablero usando la lista generada
def regla_tablero_en():
    literales = [f"En({x},{y},{n})" for x, y, n in barcos]
    return LogUtils.Ytoria_str(literales)



def reglas():
    r = []
    r.append(regla_general())

    r.append(crear_regla_4())

    r.append(regla_tablero_en())

    return LogUtils.Ytoria_str(r)





In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualizar_batalla_naval(I: dict):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(-0.5, NUM_COLUMNAS-0.5)
    ax.set_ylim(-0.5, NUM_FILAS-0.5)
    ax.set_xticks(range(NUM_COLUMNAS))
    ax.set_yticks(range(NUM_FILAS))
    ax.grid(True,linewidth=0.0)
    ax.invert_yaxis()

    colores = {
        'destapada': '#2196F3',  
        'tapada': "#FFFFFF",     
        'barco': "#720E0E"      
    }

    for x in range(NUM_FILAS):
        for y in range(NUM_COLUMNAS):
            rect = patches.Rectangle(
                (y-.5, x-0.5), 1, 1,
                facecolor=colores['tapada'],
                edgecolor='black',
                linewidth=0.5
            )
            ax.add_patch(rect)
            
    for letra, value in I.items():
        if value:
            x, y, n = parser.obtener_argumentos(letra)
            x, y, n = int(x), int(y), int(n)
            # Fondo azul para casilla destapada
            destapada_rect = patches.Rectangle(
                (y - 0.5, x - 0.5), 1, 1,
                facecolor=colores['destapada'],
                edgecolor='black',
                linewidth=0.5,
                zorder=1
            )
            ax.add_patch(destapada_rect)

    for letra, value in I.items():
        if value:
            x, y, n = parser.obtener_argumentos(letra)
            x, y, n = int(x), int(y), int(n)
            barco_rect = patches.Rectangle(
                (y - 0.5 + 0.15, x - 0.5 + 0.15), 0.7, 0.7,
                facecolor=colores['barco'],
                edgecolor='black',
                linewidth=1.0,
                zorder=2
            )
            ax.add_patch(barco_rect)
            ax.text(
                y, x, str(n),
                ha='center', va='center',
                color='white', fontsize=12,
                weight='bold',
                zorder=3
            )

    plt.title("Batalla Naval - Modo Parcial")
    plt.show()

In [ ]:
#Visualizacion del batalla naval sin completar
def barcos_a_dict(barcos):
    return {parser.parse(f"En({x},{y},{n})"): True for x, y, n in barcos}
visualizar_batalla_naval(barcos_a_dict(barcos))

In [ ]:
lt = LogicTester()
regla = reglas()
formula_lp = lt.translation_to_prover(regla)
res = lt.SATsolve(formula_lp)

print(f'\nResultado de la satisfacibilidad:\n\n\t{res}')
if res == 'UNSAT':
    print('La fórmula es insatisfacible')
else:
    print('La fórmula es satisfacible')
    
    modelo = [lt.to_numeric.literal(x) for x in res]
    modelo = [x for x in modelo if lt.to_numeric.solo_atomo(x) in lt.tseitin.atomos]
    #print(f'\nModelo:\n\n\t{modelo}')

    modelo_decodificado = [lt.to_lp.modelo_lp.decodificar(x) for x in modelo]
    print(f'\nModelo decodificado:\n\n\t{modelo_decodificado}')

    # Visualizar el modelo solucionado 
    condicion_inicial = parser.crear_interpretacion(modelo_decodificado)
    visualizar_batalla_naval(condicion_inicial)

